In [1]:
from langchain.document_loaders import TextLoader

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [3]:
# Load the text file
loader = TextLoader("ctse_notes.txt", encoding="utf-8")
documents = loader.load()

In [4]:
# Split into smaller chunks (with some overlap)
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)
chunks = splitter.split_documents(documents)

In [5]:
print(f"Total chunks created: {len(chunks)}")

Total chunks created: 301


In [6]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

In [7]:
# Load Hugging Face model
model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [8]:
# Embed and create FAISS index
db = FAISS.from_documents(chunks, model)

In [9]:
# Save for reuse (optional)
db.save_local("ctse_faiss_mini_index")

In [10]:
print("Embedding completed using MiniLM!")

Embedding completed using MiniLM!


In [11]:
#question
query = "What is microservices?"

In [12]:
# Retrieve relevant chunks
docs = db.similarity_search(query, k=3)

In [13]:
print("Top Matching Lecture Notes:")
for i, doc in enumerate(docs):
    print(f"\nResult {i+1}:\n{doc.page_content}")

Top Matching Lecture Notes:

Result 1:
new change being deployed. 
• Microservices - The microservices architecture is a design approach to build a single 
application as a set of small services with each focusing on SRP. Each service can be 
created, deployed and run independently.
• Infrastructure as Code - A practice in which infrastructure is provisioned and 
managed using code and software development techniques, such as version control 
and continuous integration.
⚬Configuration Management
⚬Policy as Code

Result 2:
Microservices: A Natural Progression
• Emergence from Real-World Use: Microservices weren’t pre-planned 
but evolved as a response to practical needs in software 
development.
• Responding to Change: Offering the agility and flexibility to adapt to 
new technologies and market demands.
Monolithic Applications
• Basic Structure
• Single-Tiered Structure: Built as a single, unified unit.
• Combined Modules: Functional modules like UI, server logic, and database interact

In [14]:
def chat(query):
    docs = db.similarity_search(query, k=3)
    print("\n Best Matches from CTSE Notes:\n")
    for i, doc in enumerate(docs):
        print(f"{i+1}. {doc.page_content}\n")

# Example
chat("Explain Cloud design patterns ")


 Best Matches from CTSE Notes:

1. •
https://www.redhat.com/en/topics/cloud-computing/what-is-paas 
•
https://www.redhat.com/en/topics/cloud-computing/what-is-saas
•
Cloud Computing: Concepts, Technology & Architecture, Thomas Erl, et al., Prentice‐Hall, 2013,
•
The Datacenter as a Computer – Designing Warehouse‐Scale Machines, 3rd Edition, Morgan & Claypool 
Publishers, 2019
•
Cloud design patterns: Prescriptive architecture guidance for cloud applications, Homer, Alex, et al. , 
2014. 
Design Patterns

2. 2014. 
Design Patterns 
A generally reusable solution to a recurring 
problem 
 
 
A template to solve the problem 
Best practices in approaching the problem 
Improve developer communication 
Cloud Application Development Issues 
Availability 
• The guaranteed proportion of time that the system 
is functional 
SLA – Service Level Agreement 
Availability (%) 
Downtime per year 
99.999
99.95
5 minutes
1 hour
99.99
99.9
99
4.4 hours
9 hours
3.7 days
Cloud Application Development Issue

In [14]:
# making a UI to the chatbot
import ipywidgets as widgets
from IPython.display import display, Markdown

In [15]:
# Create input and button
input_box = widgets.Text(
    placeholder='Ask a CTSE-related question...',
    description='You:',
    layout=widgets.Layout(width='80%')
)

submit_button = widgets.Button(
    description='Ask',
    button_style='success',
    layout=widgets.Layout(width='10%')
)

output_box = widgets.Output()

In [16]:
# Define function
def on_submit(b):
    query = input_box.value
    if not query.strip():
        return

    docs = db.similarity_search(query, k=3)
    output_box.clear_output()
    
    with output_box:
        display(Markdown(f"### 🔍 Top Matches for: `{query}`\n"))
        for i, doc in enumerate(docs):
            display(Markdown(f"**Result {i+1}:**\n{doc.page_content}\n"))

In [17]:
# Bind button click to function
submit_button.on_click(on_submit)

In [18]:
# Display all UI elements
display(widgets.HBox([input_box, submit_button]))
display(output_box)

Output()